# Tarea 4 - LTE opacities for a pure H atmosphere

We try to reproduce Figure 8.5 a, b, c, d from Gray, which show the wavelength-dependent continuous absorption coefficient $\kappa_\lambda$ for different temperatures $T$ and electron pressures $P_e$.  For simplicity, we consider only hydrogen, in the form of the neutral atom and the positive and negative ions.  Ion fractions and excitation of bound levels is calculated under the assumption of local thermodynamic equilibrium. 

## Ionization balance of hydrogen

## Excitation of bound levels of H$^0$

In [1]:
import lteopacity

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from lteopacity import LIGHTSPEED_CGS, ANGSTROM_CGS, MASA_PROTON
from lteopacity import opacidad_total, funcHden, Hplus_fraction

In [3]:
%matplotlib inline

In [5]:
wav = np.linspace(2500.0, 20000.0, 500)
nu = LIGHTSPEED_CGS / (wav * ANGSTROM_CGS)


In [6]:
# Gray, Fig 8.5a-d
subfig_list = "abcd"
T_list = [5143.0, 6429.0, 7715.0, 11572.0]
logPe_list = [1.08, 1.77, 2.50, 2.76]
for T, logPe, subfig in zip(T_list, logPe_list, subfig_list):
    Pe = 10**logPe
    Hden = funcHden(Pe, T)
    y = Hplus_fraction(Hden, T)
    for label, opacity in opacidad_total(Hden, T, nu).items():
        plt.plot(wav, opacity, label=label)
    plt.title(u"T = %i K, log Pe = %.2f, log Hden = %.2f, H+/H0 = %.2e" % (T, logPe, np.log10(Hden), y))
    plt.xlabel(u"lambda, Å")
    plt.ylabel(u"kappa_nu, unidad = cm^2/g")
    plt.axis([None, None, 0.0, None])
    plt.legend()
    plt.savefig("test_opacity_%s.png" % (subfig))
    plt.clf()